#### Importando a base de dados e as bibliotecas necessárias

In [1]:
import pandas as pd 
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000.0,3500.0
1,rtx 3060,zota galax,4000.0,4500.0


#### Criação da função para realizar a busca no Google

In [2]:
navegador = webdriver.Chrome()
def busca_google_shopping(navegador, produto, termos_banidos, preco_min, preco_max):
    navegador.get("https://www.google.com/")
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produto = produto.split(' ')
    preco_min = float(preco_min)
    preco_max = float(preco_max)
    navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
    abas_google = navegador.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for item in abas_google:
        if 'Shopping' in item.text:
            item.click()
            break
    lista_oferta = []
    lista_resultados = navegador.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')
    for resultado in lista_resultados:
        nome_produto = resultado.find_element(By.CLASS_NAME, 'Xjkr3b').text
        nome_produto = nome_produto.lower()
        tem_termos_banidos = False
        for termo in lista_termos_banidos:
            if termo in nome_produto:
                tem_termos_banidos = True
        tem_todos_termos = True
        for termo in lista_termos_produto:
            if termo not in nome_produto:
                tem_todos_termos = False
        if not tem_termos_banidos and tem_todos_termos:
            try:
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
                preco = float(preco)
            except:
                continue        
            if preco_min <= preco <= preco_max:
                nome_loja = resultado.find_element(By.CLASS_NAME, 'aULzUe')
                link_loja = nome_loja.find_element(By.XPATH, '..')
                link = link_loja.get_attribute('href')
                lista_oferta.append((nome_produto, preco, link))
    return (lista_oferta)

#### Criação de uma nova base de dados com os resultados obtidos através da busca

In [3]:
tabela_ofertas = pd.DataFrame()
for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, 'Nome']
    preco_min = tabela_produtos.loc[linha, 'Preço mínimo']
    preco_max = tabela_produtos.loc[linha, 'Preço máximo']
    termos_banidos = tabela_produtos.loc[linha, 'Termos banidos']
    lista_ofertas_google_shopping = busca_google_shopping(navegador, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_google_shopping:
        tabela_ofertas_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['Produto', 'Preço', 'Link da loja'])
        tabela_ofertas = tabela_ofertas.append( tabela_ofertas_google_shopping)
    else:
        tabela_ofertas_google_shopping = None
busca_google_shopping(navegador, produto, termos_banidos, preco_min, preco_max)

[('placa de video msi rtx 3060 12gb gddr6 gaming z trio oc 912-v390-245',
  4344.75,
  'https://www.google.com/url?url=https://www.shoptime.com.br/produto/5977515227%3Fopn%3DGOOGLEXML%26offerId%3D632e36e13c9ae19b9b9c5956%26srsltid%3DAR5OiO0a6VEHbyzJlYU6717ewd6NJ1qNuoUICdcwCiRUl6qnpC6I6ca1Iww&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwjwhZyfu9j6AhV2HLkGHfNrDrIQguUECOYO&usg=AOvVaw3OExquT5C3sttoPFz2F6Ud'),
 ('placa de vídeo nvidia geforce rtx 3060 aorus elite 12gb gddr6 gv-n3060aorus e ...',
  4379.99,
  'https://www.google.com/url?url=https://shopbetter24.com.br/shop/product/graphics-card-gigabyte-gv-n3060aorus-e-12gd-v2_369206%3Faff_t%3Dgmc%26srsltid%3DAR5OiO2-ZslCM6HFRB1QiYY4tmDCZEfhsW83UR_5KCGdC8vSVnBC0WwdfxI&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwjwhZyfu9j6AhV2HLkGHfNrDrIQguUECMQS&usg=AOvVaw3rEoYtX8ffBurZoKG5KxUR')]

#### Expotando a tabela para excel

In [4]:
display(tabela_ofertas)
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel('Ofertas.xlsx', index=False)

,Produto,Preço,Link da loja
0,iphone 12 64gb usado,3400.00,https://www.google.com/url?url=https://sp.olx....
1,"iphone 11 64gb preto, com tela de 6,1, 4g e câ...",3212.00,https://www.google.com/url?url=https://www.ext...
2,"apple iphone 12, 64 gb, verde - totalmente des...",3095.00,https://www.google.com/url?url=https://tiendam...
3,iphone 11 pro - 64 gb - 12x sem juros no cartão !,3490.00,https://www.google.com/url?url=https://pr.olx....
4,"apple iphone 12, 64 gb, (produto) vermelho - t...",3009.00,https://www.google.com/url?url=https://tiendam...
5,"vitrine: iphone 11 apple 64gb roxo tela 6,1 câ...",3099.00,https://www.google.com/url?url=https://www.car...
0,placa de video msi rtx 3060 12gb gddr6 gaming ...,4344.75,https://www.google.com/url?url=https://www.sho...
1,placa de vídeo nvidia geforce rtx 3060 aorus e...,4379.99,https://www.google.com/url?url=https://shopbet...
